# Notebook de Visualización 3D de Droplets
Este notebook explica paso a paso el pipeline para generar, exportar y visualizar el volumen de droplets a partir de datos de simulación.

## 1. Carga de datos
Ignores las líneas que comienzan con `%` y cargamos las columnas `x`, `y`, `vol_frac`.

In [ ]:
import numpy as np
# Carga los datos ignorando líneas con '%'
data = np.loadtxt('vf1_coflow_parametric.txt', comments='%')
x, y, vol_frac = data[:,0], data[:,1], data[:,2]
print(f"Datos cargados: {x.size} puntos")

## 2. Definición de grilla 2D
Creamos una grilla regular `(nx × ny)` sobre el dominio XY para interpolar los valores.

In [ ]:
from scipy.interpolate import griddata
# Parámetros de resolución
nx, ny = 200, 200
# Generación de coordenadas de la grilla
xi = np.linspace(x.min(), x.max(), nx)
yi = np.linspace(y.min(), y.max(), ny)
X, Y = np.meshgrid(xi, yi)
print(f"Grilla creada: {nx} × {ny}")

## 3. Interpolación de `vol_frac`
Interpolamos los datos dispersos sobre la grilla con método cúbico y limitamos valores a [0,1].

In [ ]:
points = np.vstack((x, y)).T
vol_frac_2d = griddata(points, vol_frac, (X, Y), method='cubic', fill_value=0.0)
vol_frac_2d = np.clip(vol_frac_2d, 0.0, 1.0)
print(f"Interpolación completa, min={vol_frac_2d.min()}, max={vol_frac_2d.max()}")

## 4. Expansión a volumen 3D
Aplicamos `repeat` para generar un volumen de `nz` slices idénticos en Z.

In [ ]:
# Número de slices en Z
nz = 100
vol_3d = np.repeat(vol_frac_2d[np.newaxis, :, :], nz, axis=0)
print(f"Volumen 3D creado: {vol_3d.shape}")

## 5. Exportación de volumen
- Guardamos RAW binario (`uint8`).
- Exportamos cada slice como PNG en la carpeta `slices/`.

In [ ]:
import os, imageio
# Convertir a uint8
vol_uint8 = (vol_3d * 255).astype(np.uint8)
# Raw
vol_uint8.tofile('droplet_volume.raw')
print('✔️ Archivo raw guardado: droplet_volume.raw')
# PNG slices
os.makedirs('slices', exist_ok=True)
for i in range(nz):
    imageio.imwrite(f'slices/slice_{i:03d}.png', vol_uint8[i])
print(f"✔️ Se exportaron {nz} slices en 'slices/'")

## 6. Visualización avanzada
### 6.1 Isosuperficie con `marching_cubes`
Extraemos una malla de nivel `vol_frac=0.5`.

In [ ]:
from skimage.measure import marching_cubes
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

level = 0.5
verts, faces, normals, values = marching_cubes(vol_3d, level=level)
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')
mesh = Poly3DCollection(verts[faces], alpha=0.7)
mesh.set_facecolor([0.7, 0.7, 1])
ax.add_collection3d(mesh)
ax.set_xlim(0, nx)
ax.set_ylim(0, ny)
ax.set_zlim(0, nz)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title(f'Isosuperficie vol_frac={level}')
plt.tight_layout()
plt.show()

### 6.2 Animación de slices Z
Creamos un GIF recorriendo capas Z con colormap `inferno`.

In [ ]:
import matplotlib.animation as animation
fig, ax = plt.subplots(figsize=(5,5))
im = ax.imshow(vol_frac_2d, cmap='inferno', origin='lower', vmin=0, vmax=1, animated=True)
ax.set_title('Slice Z=0')
def update(frame):
    im.set_array(vol_3d[frame])
    ax.set_title(f'Slice Z={frame}/{nz-1}')
    return (im,)
ani = animation.FuncAnimation(fig, update, frames=range(0, nz, max(1, nz//50)), blit=True, interval=100)
ani.save('droplet_slices.gif', writer='pillow', fps=10)
print('✔️ GIF guardado: droplet_slices.gif')